<a href="https://colab.research.google.com/github/Android998/TensorFlow2.3/blob/master/TFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Paso 1: Importar las librerias**

In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

## **Paso 2:Pre procesado de datos**

In [2]:
drive.mount("/content/drive")
base_dir = "./drive/My Drive/object_photos"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 256
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)


In [4]:
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

print(len(train_generator))

Found 38681 images belonging to 40 classes.
152


In [5]:
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')
print(len(val_generator))

Found 9652 images belonging to 40 classes.
38


### **Generamos el archivo labels con las etiquetas de los objetos**

In [6]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'backpack': 0, 'bed': 1, 'belt': 2, 'blind stick': 3, 'bottle': 4, 'carpet': 5, 'cellular phone': 6, 'chair': 7, 'cup-glass': 8, 'door': 9, 'eyeglasses': 10, 'fan': 11, 'handkerchief': 12, 'headphones': 13, 'keys': 14, 'laptop': 15, 'mouse': 16, 'newspaper': 17, 'notebook': 18, 'pen': 19, 'pencil': 20, 'pillow': 21, 'plate': 22, 'plug': 23, 'printer': 24, 'radiator': 25, 'remote control': 26, 'shoes': 27, 'shorts': 28, 'socks': 29, 't-shirt': 30, 'table': 31, 'television': 32, 'toothbrush': 33, 'towel': 34, 'trash bin': 35, 'wallet': 36, 'wardrobe': 37, 'watch': 38, 'window (shade)': 39}


In [7]:
!cat labels.txt

backpack
bed
belt
blind stick
bottle
carpet
cellular phone
chair
cup-glass
door
eyeglasses
fan
handkerchief
headphones
keys
laptop
mouse
newspaper
notebook
pen
pencil
pillow
plate
plug
printer
radiator
remote control
shoes
shorts
socks
t-shirt
table
television
toothbrush
towel
trash bin
wallet
wardrobe
watch
window (shade)

## **Paso 3: Crear el modelo base ya entrenado**

In [8]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [9]:
modelo = tf.keras.models.Sequential()
modelo.add(tf.keras.layers.InputLayer(input_shape=IMG_SHAPE))
modelo.add(tf.keras.layers.ZeroPadding2D())
modelo.add(tf.keras.layers.Conv2D(32, 3, activation='relu', padding="same"))
modelo.add(tf.keras.layers.Dropout(0.2))
modelo.add(tf.keras.layers.BatchNormalization())
modelo.add(tf.keras.layers.Conv2D(64, 3, activation='relu', padding="same"))
modelo.add(tf.keras.layers.Conv2D(64, 3, activation='relu', padding="same"))
modelo.add(tf.keras.layers.BatchNormalization())
modelo.add(tf.keras.layers.BatchNormalization())
modelo.add(tf.keras.layers.Conv2D(64, 3, activation='relu', padding="same"))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(4, 4)))
modelo.add(tf.keras.layers.Dropout(0.2))
modelo.add(tf.keras.layers.BatchNormalization())
modelo.add(tf.keras.layers.Conv2D(128, 3, activation='relu', padding="same"))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
modelo.add(tf.keras.layers.Dropout(0.2))
modelo.add(tf.keras.layers.BatchNormalization())
modelo.add(tf.keras.layers.Conv2D(64, 3, activation='relu', padding="same"))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
modelo.add(tf.keras.layers.Dropout(0.2))
modelo.add(tf.keras.layers.BatchNormalization())
modelo.add(tf.keras.layers.Flatten())
modelo.add(tf.keras.layers.Dropout(0.2))
modelo.add(tf.keras.layers.Dense(units=128, activation="relu"))
modelo.add(tf.keras.layers.Dropout(0.2))
modelo.add(tf.keras.layers.BatchNormalization())
modelo.add(tf.keras.layers.Dense(units=256, activation="relu"))
modelo.add(tf.keras.layers.Dense(units=40, activation="softmax"))

In [10]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 226, 226, 32)      896       
_________________________________________________________________
dropout (Dropout)            (None, 226, 226, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 226, 226, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 226, 226, 64)      18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 226, 226, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 226, 226, 64)      2

In [11]:
modelo.compile(optimizer=tf.keras.optimizers.Adam(), 
               loss='categorical_crossentropy', 
               metrics=['accuracy'])

In [ ]:
history = modelo.fit(train_generator, 
           steps_per_epoch=len(train_generator), 
           epochs=10,
           validation_data=val_generator, 
           validation_steps=len(val_generator))

Epoch 1/10


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("fashion_model.h5")

In [ ]:
files.download("fashion_model.json")
files.download("fashion_model.h5")